# Neon CRM SDK - Search and Output Fields Examples

This notebook demonstrates how to retrieve available search fields and output fields for different Neon CRM resources. This is essential for building dynamic search queries and understanding what data can be retrieved from each endpoint.

## Overview

Each Neon CRM resource has:
- **Search Fields**: Fields that can be used to filter/search records
- **Output Fields**: Fields that can be included in the response data

Both include standard fields (built into Neon) and custom fields (defined by your organization).

In [1]:
import os
from typing import Any, Dict

from neon_crm import NeonClient


# Helper function to display field information in a clean format
def display_fields(
    resource_name: str, fields_data: Dict[str, Any], field_type: str = "search"
):
    print(f"\n=== {resource_name.upper()} {field_type.upper()} FIELDS ===")

    # Display standard fields
    standard_fields = fields_data.get("standardFields", [])
    if standard_fields:
        print(f"\nStandard Fields ({len(standard_fields)}):")
        if isinstance(standard_fields[0], str):
            # Output fields are sometimes just strings
            for field in sorted(standard_fields):
                print(f"  • {field}")
        else:
            # Search fields have more structure
            for field in sorted(standard_fields, key=lambda x: x.get("fieldName", "")):
                name = field.get("fieldName", "")
                field_type_info = field.get("fieldType", "")
                field_operators_info = field.get("operators", "")
                print(
                    f"  • {name} ({field_type_info}) operators: {field_operators_info}"
                )

    # Display custom fields
    custom_fields = fields_data.get("customFields", [])
    if custom_fields:
        print(f"\nCustom Fields ({len(custom_fields)}):")
        for field in sorted(custom_fields, key=lambda x: x.get("displayName", "")):
            name = field.get("displayName", "")
            field_id = field.get("id", "")
            field_type_info = field.get("fieldType", "")
            field_operators_info = field.get("operators", "")
            print(
                f"  • {name} (ID: {field_id}, Type: {field_type_info} operators: {field_operators_info}"
            )

    if not standard_fields and not custom_fields:
        print("  No fields found")

    print(f"\nTotal: {len(standard_fields) + len(custom_fields)} fields")

In [2]:
# Initialize the Neon CRM client
# Option 1: Use environment variables (recommended)
client = NeonClient(
    org_id=os.getenv("NEON_ORG_ID"),
    api_key=os.getenv("NEON_API_KEY"),
    environment="production",  # or "trial"
)

# Option 2: Set credentials directly (not recommended for production)
# client = NeonClient(
#     org_id="your_org_id_here",
#     api_key="your_api_key_here",
#     environment="production"
# )

print("Neon CRM client initialized successfully!")

Neon CRM client initialized successfully!


## 1. Accounts Resource

Accounts are the core records in Neon CRM representing people and organizations.

In [ ]:
# Get search fields for accounts
try:
    search_fields = client.accounts.get_search_fields()
    display_fields("Accounts", search_fields, "search")
except Exception as e:
    print(f"Error getting account search fields: {e}")

# Get output fields for accounts
try:
    output_fields = client.accounts.get_output_fields()
    display_fields("Accounts", output_fields, "output")
except Exception as e:
    print(f"Error getting account output fields: {e}")

## 2. Donations Resource

Donations represent financial contributions from accounts.

In [ ]:
# Get search and output fields for donations
try:
    search_fields = client.donations.get_search_fields()
    display_fields("Donations", search_fields, "search")

    output_fields = client.donations.get_output_fields()
    display_fields("Donations", output_fields, "output")
except Exception as e:
    print(f"Error getting donation fields: {e}")

## 3. Events Resource

Events represent fundraising events, meetings, and other activities.

In [ ]:
# Get search and output fields for events
try:
    search_fields = client.events.get_search_fields()
    display_fields("Events", search_fields, "search")

    output_fields = client.events.get_output_fields()
    display_fields("Events", output_fields, "output")
except Exception as e:
    print(f"Error getting event fields: {e}")

## 4. Activities Resource

Activities track interactions and communications with accounts.

In [ ]:
# Get search and output fields for activities
try:
    search_fields = client.activities.get_search_fields()
    display_fields("Activities", search_fields, "search")

    output_fields = client.activities.get_output_fields()
    display_fields("Activities", output_fields, "output")
except Exception as e:
    print(f"Error getting activity fields: {e}")

## 6. Practical Usage Examples

Now that we know what fields are available, let's see how to use them in actual search queries.

In [ ]:
# Example 1: Search accounts by name using discovered fields
print("=== EXAMPLE 1: Account Search ===")
try:
    # Use field names we discovered above
    search_request = {
        "searchFields": [
            {"field": "First Name", "operator": "CONTAIN", "value": "John"}
        ],
        "outputFields": ["Account ID", "First Name", "Last Name", "Email 1"],
        "pagination": {"currentPage": 0, "pageSize": 5},
    }

    # search() returns a generator of individual account dictionaries
    results = list(client.accounts.search(search_request))
    print(f"Found {len(results)} accounts with 'John' in first name")

    for account in results[:3]:  # Show first 3
        print(
            f"  • {account.get('First Name', '')} {account.get('Last Name', '')} ({account.get('Email 1', 'No email')})"
        )

except Exception as e:
    print(f"Error searching accounts: {e}")

In [ ]:
# Example 2: Search donations by date range (using a string field with EQUAL operator)
print("\n=== EXAMPLE 2: Donation Search ===")
try:
    search_request = {
        "searchFields": [
            {
                "field": "Donation Date",  # Use date field instead
                "operator": "GREATER_THAN",  # Date fields support numeric operators
                "value": "2024-01-01",  # Search for donations after Jan 1, 2024
            }
        ],
        "outputFields": [
            "Donation ID",
            "Account ID",
            "Donation Amount",
            "Donation Date",
            "Campaign Name",  # Correct field name for campaign
        ],
        "pagination": {"currentPage": 0, "pageSize": 5},
    }

    # search() returns a generator of individual donation dictionaries
    results = list(client.donations.search(search_request))
    print(f"Found {len(results)} donations after 2024-01-01")

    for donation in results[:3]:  # Show first 3
        amount = donation.get("Donation Amount", 0)
        date = donation.get("Donation Date", "Unknown")
        campaign = donation.get("Campaign Name", "No campaign")
        print(f"  • ${amount} on {date} - {campaign}")

except Exception as e:
    print(f"Error searching donations: {e}")

In [3]:
# Example 3: Search accounts by name using discovered fields, but without setting outputFields
print("=== EXAMPLE 3: Account Search ===")
try:
    # Use field names we discovered above
    search_request = {
        "searchFields": [
            {"field": "First Name", "operator": "CONTAIN", "value": "John"}
        ],
        "pagination": {"currentPage": 0, "pageSize": 5},
    }

    # search() returns a generator of individual account dictionaries
    results = list(client.accounts.search(search_request))
    print(f"Found {len(results)} accounts with 'John' in first name")

    for account in results[:3]:  # Show first 3
        print(
            f"  • {account.get('First Name', '')} {account.get('Last Name', '')} ({account.get('Email 1', 'No email')})"
        )

except Exception as e:
    print(f"Error searching accounts: {e}")

=== EXAMPLE 3: Account Search ===
Error searching accounts: HTTP 400: [Code 1061] Output fields cannot be greater than 300.


## 7. Tips and Best Practices

### Dynamic Field Discovery
- Always call `get_search_fields()` and `get_output_fields()` to get the most current field list
- Custom fields vary between organizations, so discovery is essential for portable code
- Cache field information when possible to reduce API calls

### Field Types and Operators
- String fields: Use `CONTAIN`, `EQUAL`, `NOT_EQUAL`, `BLANK`, `NOT_BLANK`
- Number fields: Add `GREATER_THAN`, `LESS_THAN`, `IN_RANGE` operators
- Date fields: Support all number operators plus date-specific formatting
- Boolean fields: Limited to `EQUAL` and `NOT_EQUAL`

### Error Handling
- Some resources might not support field discovery
- Always wrap API calls in try-catch blocks
- The SDK provides helpful suggestions for misspelled field names

In [ ]:
# Example 3: Comprehensive field discovery for all major resources
print("=== SUMMARY: Field Counts by Resource ===")

resources_to_check = [
    ("accounts", client.accounts),
    ("donations", client.donations),
    ("events", client.events),
    ("activities", client.activities),
    ("memberships", client.memberships),
]

summary_data = []

for resource_name, resource_obj in resources_to_check:
    try:
        search_fields = resource_obj.get_search_fields()
        output_fields = resource_obj.get_output_fields()

        search_count = len(search_fields.get("standardFields", [])) + len(
            search_fields.get("customFields", [])
        )
        output_count = len(output_fields.get("standardFields", [])) + len(
            output_fields.get("customFields", [])
        )

        summary_data.append(
            {
                "resource": resource_name.title(),
                "search_fields": search_count,
                "output_fields": output_count,
            }
        )

    except Exception as e:
        summary_data.append(
            {
                "resource": resource_name.title(),
                "search_fields": f"Error: {e}",
                "output_fields": f"Error: {e}",
            }
        )

# Display summary table
print(f"{'Resource':<12} {'Search Fields':<15} {'Output Fields':<15}")
print("-" * 45)
for data in summary_data:
    print(
        f"{data['resource']:<12} {data['search_fields']:<15} {data['output_fields']:<15}"
    )

print(
    "\nThis information helps you understand the scope of searchable and retrievable data for each resource type."
)